
# Embeddings - Vetorização de Texto

Gera vetores densos a partir de frases em português usando o endpoint `text-embedding-3-large`.

In [1]:

import os
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
sentences = [
    'O laboratório de dados inaugura nova área de pesquisa.',
    'As equipes de analytics colaboram em painéis interativos.',
    'Planos de expansão internacional estão em pauta este ano.'
]

emb = client.embeddings.create(
    model='text-embedding-3-large',
    input=sentences
)

for text, vector in zip(sentences, emb.data):
    print(f'Frase: {text}')
    print(f'Dimensão do embedding: {len(vector.embedding)}')
    print(f'Primeiros valores: {vector.embedding[:5]}')
    print('-' * 40)


Frase: O laboratório de dados inaugura nova área de pesquisa.
Dimensão do embedding: 3072
Primeiros valores: [-0.019725626334547997, 0.009618226438760757, -0.00916871428489685, 0.014873545616865158, 0.005427191499620676]
----------------------------------------
Frase: As equipes de analytics colaboram em painéis interativos.
Dimensão do embedding: 3072
Primeiros valores: [-0.01589900441467762, -0.009152169339358807, -0.021542292088270187, 0.0009127389639616013, 0.031269364058971405]
----------------------------------------
Frase: Planos de expansão internacional estão em pauta este ano.
Dimensão do embedding: 3072
Primeiros valores: [-0.01275450550019741, 0.025429129600524902, -0.021528221666812897, 0.037890736013650894, 0.03413627669215202]
----------------------------------------



## Observações

- Normalize ou armazene os vetores em um banco vetorial (FAISS, Pinecone, Weaviate).
- Para buscas semânticas, calcule a similaridade coseno entre embeddings.
- Considere reduzir dimensionalidade ou quantizar para economia de armazenamento.
- Mantenha metadados associados a cada vetor (texto original, idioma, data).



## Caso prático: FAQ inteligente

Simula uma central de ajuda. Vetorizamos perguntas frequentes e comparamos a dúvida do usuário para sugerir o artigo mais relevante.

In [2]:
faq_base = [
    {
        'question': 'Como faço para redefinir minha senha?',
        'answer': 'Acesse o portal, clique em "Esqueci minha senha" e siga o passo a passo enviado por e-mail.'
    },
    {
        'question': 'Quais são as opções de pagamento disponíveis?',
        'answer': 'Você pode pagar via cartão de crédito, boleto ou PIX. Consulte a área de faturamento para parcelamentos.'
    },
    {
        'question': 'Como acompanho a entrega do meu pedido?',
        'answer': 'No painel "Pedidos", clique em "Detalhes" e acompanhe o código de rastreio atualizado em tempo real.'
    }
]

faq_embeddings = client.embeddings.create(
    model='text-embedding-3-large',
    input=[item['question'] for item in faq_base]
)
for item, emb_item in zip(faq_base, faq_embeddings.data):
    item['vector'] = emb_item.embedding

user_question = 'Esqueci minha senha do portal da empresa. Como recupero o acesso?'
user_embedding = client.embeddings.create(
    model='text-embedding-3-large',
    input=user_question
)
user_vector = user_embedding.data[0].embedding

def cosine_similarity(vec_a, vec_b):
    from math import sqrt
    dot = sum(a * b for a, b in zip(vec_a, vec_b))
    norm_a = sqrt(sum(a * a for a in vec_a))
    norm_b = sqrt(sum(b * b for b in vec_b))
    return dot / (norm_a * norm_b)

best_item = max(
    faq_base,
    key=lambda item: cosine_similarity(user_vector, item['vector'])
)

print('Pergunta do usuário:', user_question)
print('Artigo sugerido:', best_item['question'])
print('Resposta sugerida:', best_item['answer'])


Pergunta do usuário: Esqueci minha senha do portal da empresa. Como recupero o acesso?
Artigo sugerido: Como faço para redefinir minha senha?
Resposta sugerida: Acesse o portal, clique em "Esqueci minha senha" e siga o passo a passo enviado por e-mail.
